In [32]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
!rm output/dynamic_delta_fits/??/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [33]:
iOption0 = 1
is_sigmalised = 1
sim = 0
new = 1

Min_Entries = 100

In [34]:
isigma = 2
N_files = 12
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if new: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/rungroup/"
file_names="my-12th_QA_19423_1078runs_rg" #my-25th_QA_19197_1000runs
if not new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [35]:
if sim :
    file_path="../sim/output/"
    file_names="my-kek"

    #phi_bins = [96,97,98,99,100,100,101,101,102,103,104,105]

In [36]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg0.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg1.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg2.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg3.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg4.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg5.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg6.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg7.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg8.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg9.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg10.root
input/NewHadron/rungroup/my-12th_QA_19423_1078runs_rg11.root


In [37]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.35, 0.65, 1.0, 1.4, 1.8, 2.2, 2.7, 3.4, 4.35]) 
pt_boarders =  array('d', [0.2, 0.5, 0.8, 1.2, 1.6, 2.0, 2.4, 3.0, 3.8, 4.9])


In [38]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8

In [39]:
def emc_fitter(InHist, iphi, seed=1, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if new: central_bean_prev = (0.002*phi_bins[2*iphi]-0.001-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(ilayer)+"/"+str(iphi)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        g_gaus2.SetParameter(1,0.5*(boards_fit2[iilayer][0]+boards_fit2[iilayer][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[iilayer][1]-boards_fit2[iilayer][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/100,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [40]:
def emc_centr_fitter(ilayer=1,InHist=hists_emc_read, phibins=[0,1,2,3,4,5], sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for iFile in range(N_files):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(emc_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [41]:
#Ntr = N_files
Ntr = N_layers
pool = mp.Pool(Ntr)
#output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
with pool:
    output_array = pool.map(emc_centr_fitter, [i for i in range(Ntr)])
pool.close()

In [42]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(N_files):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [43]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [44]:
g_sigma = []
for ifile in range(N_files):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    if N_files>1:
        c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",225*N_layers,1900)
        c3.Divide(int(N_layers/4),4)
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        g_sigma_layer = []
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

In [45]:
print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_ilayer in fit_functions_ifile:
        print("{")
        for fits in fit_functions_ilayer:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[12][16][6][3]={
{
{
{0.134795, 1.068835, -0.128824},
{0.015665, 1.163528, -0.09662},
{0.0, 1.44547, -0.337255},
{0.786801, 0.391052, -0.197261},
{0.0, 1.551666, -0.441617},
{1.052721, 0.24378, -1.730925},
},
{
{0.0, 1.631465, -0.475462},
{0.0, 1.777966, -0.599601},
{0.0, 1.675191, -0.464398},
{1e-06, 1.509253, -0.316503},
{1.102108, 0.153521, -1.841787},
{1e-06, 1.510878, -0.352244},
},
{
{0.02366, 1.081146, -0.100273},
{0.745028, 0.270293, 0.0249},
{0.920417, 1e-05, 10.22999},
{4e-06, 1.178184, -0.168326},
{0.003451, 1.084402, -0.086872},
{0.005046, 1.083088, -0.079391},
},
{
{0.941234, 0.173923, -1.134829},
{0.000383, 1.119319, -0.142364},
{0.768855, 0.252953, -0.010491},
{0.873138, 0.208055, -0.497536},
{0.242139, 0.82065, -0.079541},
{0.9638, 0.14093, -1.968859},
},
{
{0.004581, 1.042408, -0.06017},
{0.730167, 0.238941, 0.055522},
{8.4e-05, 1.064969, -0.104071},
{0.008559, 1.027626, -0.062448},
{0.001349, 1.086804, -0.101866},
{0.03215, 1.062635, -0.09296},
},
{
{2e

In [46]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0<2:
                    fit_functions1[-1].SetParameter(2,-1)
                    fit_functions1[-1].SetParLimits(2,-1.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-20,20)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,-20,20)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    
                    fit_functions1[-1].FixParameter(2,-1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [47]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    if N_files>1:
        c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",225*N_layers,1900)
        c3.Divide(int(N_layers/4),4)
    g_mean = [[[[]]*N_phi]*N_layers]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean[ifile][ilayer][iphi].append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            
            Format_Graph(g_mean[ifile][ilayer][iphi][-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            g_mean[ifile][ilayer][iphi][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.4,2)
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][ilayer][iphi][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [48]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

const float the_sMean_pt_params[12][16][3]={
{
{-0.108295, 0.058274, -1.0},
{-0.055493, -0.455538, -1.0},
{-0.137187, 0.372019, -1.0},
{0.131067, -0.119208, -1.0},
{0.209046, 0.148207, -1.0},
{0.194897, -0.461304, -1.0},
{-0.000363, 0.187042, -1.0},
{0.204607, -0.311906, -1.0},
{-0.145736, 0.396475, -1.0},
{0.259943, -0.67375, -1.0},
{-0.134212, 0.331147, -1.0},
{0.211531, -0.4507, -1.0},
{0.036138, -0.199106, -1.0},
{0.247724, -0.486623, -1.0},
{-0.183864, 0.243683, -1.0},
{0.194895, -0.365068, -1.0},
},
{
{-0.01356, -0.124204, -1.0},
{-0.141069, -0.191877, -1.0},
{-0.001845, 0.053255, -1.0},
{0.191263, -0.256284, -1.0},
{0.222618, 0.08833, -1.0},
{0.168782, -0.31703, -1.0},
{0.002279, 0.163905, -1.0},
{0.291426, -0.50024, -1.0},
{-0.017609, 0.0642, -1.0},
{0.219343, -0.493041, -1.0},
{-0.197852, 0.409976, -1.0},
{0.187776, -0.434723, -1.0},
{-0.049309, -6.4e-05, -1.0},
{0.246957, -0.458047, -1.0},
{-0.13936, 0.216888, -1.0},
{0.263328, -0.582152, -1.0},
},
{
{-0.175658, 0.263618, -1.

In [49]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dphi1 = array( 'd' )
dphi_err1 = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
dynamic_dphi1 = []
dynamic_dphi_err1 = []
for iphi in range(N_phi):
    if not is_sigmalised and not new: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    if new and not is_sigmalised: dphi.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
    if is_sigmalised: 
        dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
        dphi1.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)
    dphi_err1.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    dynamic_dphi_ifile1 = []
    dynamic_dphi_err_ifile1 = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        dynamic_dphi_ilayer1 = array( 'd' )
        dynamic_dphi_err_ilayer1 = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(0))
            dynamic_dphi_ilayer1.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(1))
            dynamic_dphi_err_ilayer1.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(1))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
        dynamic_dphi_ifile1.append(dynamic_dphi_ilayer1)
        dynamic_dphi_err_ifile1.append(dynamic_dphi_err_ilayer1)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)
    dynamic_dphi1.append(dynamic_dphi_ifile1)
    dynamic_dphi_err1.append(dynamic_dphi_err_ifile1)

print("dphi = ", dphi)
print(dynamic_dphi)
print(dynamic_dphi1)

dphi =  array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1])
[[array('d', [0.30629481806380143, 0.2592802317110033, 0.4998772598664372, -0.10829544818732564, -0.409214244694992, -0.35659163429374924]), array('d', [0.13020653836201182, -0.12144930696442681, -0.05549320531911962, -0.3660796142309122, -0.49999987695640635, -0.49999999904145387]), array('d', [0.4238707744264844, 0.20594819841317266, 0.25112270989674507, -0.13718705795685535, -0.39223661237683705, -0.3547633441260536]), array('d', [0.46749751039596166, 0.283906517091557, 0.13106704493858712, -0.028451305679759717, -0.26015359909066943, -0.45569352066356833]), array('d', [0.49999999997539085, 0.49999999920752813, 0.49999936646229215, 0.20904640433451221, -0.19736810229299556, -0.27380963066990704]), array('d', [0.4999969434208616, 0.26598178931871236, 0.19489734623268576, -0.08528701229807092, -0.49999996398626245, -0.49999998558594466]), array('d', [0.4999999728635269, 0.4999999901991181, 0.353949264632348, -0.00036315818554983, -

In [50]:
g_mean_phi = []
for ifile in range(N_files):
    c4 = root.TCanvas(f"c4{ifile}",f"c4{ifile}",600*2,450)
    c4.Divide(2,1)
    g_mean_phi_type=[]
    h3=[]
    for itype in range(2):
        g_mean_phi_layer=[]
        c4.cd(itype+1)
        min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
        if is_sigmalised==1: max, min, x0, x1 = 1+itype*2, -1-itype*2, -2, 5
        names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
        legends =[]
        for iphi in range(1):
            h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
            h3[-1].Draw()
            legends.append(root.TLegend(0.55,0.4,0.85,0.9))
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            for ilayer in range(0,N_layers):

                if itype==0:                
                    g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
                    g_mean_phi_layer[-1].SetName(f"test_graph_{itype}_{ifile}_{ilayer}_{iphi}_{iOption0}")
                else:
                    g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi1, dynamic_dphi1[ifile][ilayer],dphi_err1,dynamic_dphi_err1[ifile][ilayer]))

                Format_Graph(g_mean_phi_layer[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
                g_mean_phi_layer[-1].Fit("pol1","Q","")
                g_mean_phi_layer[-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
                g_mean_phi_layer[-1].Draw('sameP')
                legends[-1].AddEntry(g_mean_phi_layer[-1],layers[ilayer],"p")

            legends[-1].Draw()
        g_mean_phi_type.append(g_mean_phi_layer)
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_phi_{ifile}{iOption0}{is_sigmalised}.png")
    g_mean_phi.append(g_mean_phi_type)
print(g_mean_phi[0][0][0].GetFunction("pol1").GetParameter(0),g_mean_phi[0][1][0].GetFunction("pol1").GetParameter(0))


-0.04115414602639007 -0.4202167199788906


In [51]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [0.962587021851784, 0.9518890164182384, 1.0338836763168406, 0.9661163236831595, 1.0534140229209554, 0.9781328974279627]), array('d', [0.9832281796487659, 1.0070332579797647, 1.0151808349911806, 0.9848191650088194, 0.8891786085500704, 0.9684179827749618]), array('d', [1.0491846789078463, 1.0160412011748312, 0.9197480762990006, 1.0802519237009993, 1.0390331151221264, 1.0432448614890268]), array('d', [1.0115946255495003, 1.014464395812913, 0.9927584611152956, 1.0072415388847045, 1.0027862504255045, 0.9888421696351496]), array('d', [1.0082065953519197, 0.9682589135580174, 1.0032048304432828, 0.9967951695567171, 1.0261554458358642, 1.0383989573156203]), array('d', [1.0310175831962876, 0.9752337533922396, 1.0487622671703096, 0.9512377328296904, 0.9446319062196337, 0.9555189221654342]), array('d', [1.0130318897685573, 1.0087846055074452, 0.9905738124890562, 1.009426187510944, 1.0288578943083653, 1.0711446215114608]), array('d', [0.3975073931770207, 0.340817690379128, 1.6606845107

In [52]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer%4])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/sigma_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [53]:
if False:
    print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
    for fit_functions_ifile in fit_functions:
        print("    {")
        for fits in fit_functions_ifile:
            print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
        print("    },")      
    print("};")
    print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
    for ifile in range(N_files):
        print("    {")
        for ilayer in range(N_layers):
            print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
        print("    },")
    print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params0[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")

print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params1[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][1][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][1][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")

const float the_mean_the_params0[12][16][2]=
{
    {
        {-0.041154, -0.342449},
        {-0.244938, -0.250392},
        {-0.021539, -0.365148},
        {-0.001491, -0.314093},
        {0.170816, -0.340835},
        {-0.063972, -0.424317},
        {0.033022, -0.47647},
        {-0.003337, -0.388964},
        {-0.080132, -0.283361},
        {0.02885, -0.348527},
        {-0.095321, -0.442821},
        {0.031462, -0.323054},
        {0.009505, -0.243056},
        {0.018597, -0.327898},
        {-0.117837, -0.35077},
        {0.022967, -0.30914},
    },
    {
        {-0.000553, -0.277139},
        {-0.253661, -0.242879},
        {0.019482, -0.280452},
        {0.032425, -0.279004},
        {0.161897, -0.332049},
        {-0.031428, -0.434486},
        {0.052436, -0.461506},
        {0.023526, -0.493993},
        {-0.038422, -0.3259},
        {0.036361, -0.342621},
        {-0.089386, -0.351871},
        {0.023221, -0.3099},
        {-0.016338, -0.392675},
        {0.046999, -0.37713}

In [54]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()